In [1]:
import numpy as np
import pandas as pd
import time
import itertools
import random

IP=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] #inventory 
AI=[0,1,2,3,4,5] # age 
state_space=[IP,AI]
combination = [k for k in itertools.product(*state_space)]

States = pd.DataFrame(combination, columns = ['Inventory', 'Age'])
Remaining_lifeunits=States['Inventory']*States['Age']


States['Remaining_lifeunits']= Remaining_lifeunits
print('\nStates :\n', States)                    #possible states...defined by (total Inventory, total Remaining_lifeunits)
Actions=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]




States :
      Inventory  Age  Remaining_lifeunits
0            0    0                    0
1            0    1                    0
2            0    2                    0
3            0    3                    0
4            0    4                    0
..         ...  ...                  ...
121         20    1                   20
122         20    2                   40
123         20    3                   60
124         20    4                   80
125         20    5                  100

[126 rows x 3 columns]


In [2]:
Q=np.zeros((len(States),len(Actions))) 
print(Q)
Q.shape

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


(126, 21)

In [3]:
#Parameters
m=5 #product lifetime
Co=1
Cs=2
l=0

#Algorithm parameters
Max_steps=50 
total_episodes=2500
l_rate = 0.80 
gamma=0.97
epsilon=0.6
alpha=0.56

#Function to choose the next action 
def choose_action(state1): 
    if state1['Remaining_lifeunits'].sum() ==0: #checking all rows for different lot's remaining lifeunits
        action1=20
    else: 
        action1=random.choice(Actions)
    return action1 

  

In [5]:
#Initializing the reward 
Rewards=[] #for total episodes
  
# Starting the Q learning 
for episode in range(total_episodes): 
    state1_ini= States.loc[125,:]                             #1.initial state with lead time of 2 timesteps
    print(state1_ini)
    
    for i in range(1,6,1): 
         
        d1=np.random.gamma(4,5,1) 
        D1=int(d1)                                              #2. demand 
        print(D1)
        
        TS = max((D1-state1_ini['Inventory'].sum()),0)          #shortage quantity
                                                               
        inv_remained = max((state1_ini['Inventory']-D1),0)      #3.1 old lot...remained status after satisfying demand
        age_remained=state1_ini['Age'] - 1                                      
        lifeunits_remained=inv_remained *age_remained     
       
        state1 = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits']) 
        state1.loc[0] = [inv_remained,age_remained,lifeunits_remained]
        print(state1)                                           #state 1....for taking action 1
        
        outdated=state1[state1['Age']==0]
        print('\nOutdated dataframe :\n', outdated)             #outdated quantity     
        TO=outdated['Inventory'].sum()
        print(TO)
        
        reward=-(Co*TO + Cs*TS)                                 #reward function for each timestep
        print('\nreward :\n', reward)
        
        action1 = choose_action(state1)                        # choosing action ..see States vs state 
        
        
        inv_updated= action1                                   #3.2 new lot..updated status after ordering         
        age_updated= 5                                   
        lifeunits_updated=inv_updated * age_updated
        state2_ini = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits']) 
        state2_ini.loc[0] = state1.loc[0]
        state2_ini.loc[1] = [inv_updated,age_updated,lifeunits_updated]
        print('\nstate2 initia lots :\n', state2_ini) 
        
        
        d2=np.random.gamma(4,5,1) 
        D2=int(d2)                                              
        print(D2)
        TS = max((D2-state2_ini['Inventory'].sum()),0)        
        
        state2 = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits'])
        state2.loc[0,'Inventory']= max((state2_ini.loc[0,'Inventory']-D2),0) 
        
        if state2_ini.loc[0,'Inventory']-D2<0:
            state2.loc[1,'Inventory']= max((state2_ini.loc[1,'Inventory']-(D2 - state2_ini.loc[0,'Inventory'])),0)
        else :
            state2.loc[1,'Inventory']=state2_ini.loc[1,'Inventory']
            
        state2.loc[0,'Age']= state2_ini.loc[0,'Age']-1 
        state2.loc[1,'Age']= state2_ini.loc[1,'Age']-1
        state2.loc[0,'Remaining_lifeunits']= state2.loc[0,'Inventory']*state2.loc[0,'Age']
        state2.loc[1,'Remaining_lifeunits']= state2.loc[1,'Inventory']*state2.loc[1,'Age']
                              
        print('\nstate2 lots :\n', state2)                     #state 2...for taking action 2  
        
        outdated=state2[state2['Age']==0]
        print('\nOutdated dataframe :\n', outdated)                
        TO=outdated['Inventory'].sum()
        print(TO)
        
        reward=-(Co*TO + Cs*TS)                              
        print('\nreward :\n', reward)
        action2 = choose_action(state2)                        
        
        
        #4. Q value update
        predict = Q[state1['Remaining_lifeunits'].sum(), action1]
        target = reward + gamma * np.max(Q[state2['Remaining_lifeunits'].sum(), :])
        Q[state1['Remaining_lifeunits'].sum(), action1] = Q[state1['Remaining_lifeunits'].sum(), action1] + alpha * (target - predict)
 
        print(Q[state1['Remaining_lifeunits'].sum(), action1])
        

        
        inv_updated= action2                                        
        age_updated= 5                                   
        lifeunits_updated=inv_updated * age_updated
        state3_ini = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits']) 
        state3_ini.loc[0] = state2.loc[0]
        state3_ini.loc[1] = state2.loc[1]
        state3_ini.loc[2] = [inv_updated,age_updated,lifeunits_updated]
        print('\nstate3 initia lots :\n', state3_ini) 
              

        d3=np.random.gamma(4,5,1) 
        D3=int(d3)                                              
        print(D3)
        TS = max((D2-state3_ini['Inventory'].sum()),0)        
        
        state3 = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits'])
        state3.loc[0,'Inventory']= max((state3_ini.loc[0,'Inventory']-D3),0)  
        
        if state3_ini.loc[0,'Inventory']-D3<0:
            state3.loc[1,'Inventory']= max((state3_ini.loc[1,'Inventory']-(D3 - state3_ini.loc[0,'Inventory'])),0)
        else :
            state3.loc[1,'Inventory']=state3_ini.loc[1,'Inventory']
            
        if state3_ini.loc[1,'Inventory']-(D3 - state3_ini.loc[0,'Inventory'])<0: 
            state3.loc[2,'Inventory']= max((state3_ini.loc[2,'Inventory']-(D3 - (state3_ini.loc[1,'Inventory']+
                                                                             state3_ini.loc[0,'Inventory']))),0)
        else:    
            state3.loc[2,'Inventory']=state3_ini.loc[2,'Inventory']
            
        
        state3.loc[0,'Age']= state3_ini.loc[0,'Age']-1 
        state3.loc[1,'Age']= state3_ini.loc[1,'Age']-1
        state3.loc[2,'Age']= state3_ini.loc[2,'Age']-1
        state3.loc[0,'Remaining_lifeunits']= state3.loc[0,'Inventory']*state3.loc[0,'Age']
        state3.loc[1,'Remaining_lifeunits']= state3.loc[1,'Inventory']*state3.loc[1,'Age']
        state3.loc[2,'Remaining_lifeunits']= state3.loc[2,'Inventory']*state3.loc[2,'Age']
                              
        print('\nstate3 lots :\n', state3)                     #state 3...for taking action 3  
        
        outdated=state3[state3['Age']==0]
        print('\nOutdated dataframe :\n', outdated)             
        TO=outdated['Inventory'].sum()
        print(TO)
        
        reward=-(Co*TO + Cs*TS)                              
        print('\nreward :\n', reward)
        action3 = choose_action(state3)     
        
        predict = Q[state2['Remaining_lifeunits'].sum(), action2]             #4. Q value update
        target = reward + gamma * Q[state3['Remaining_lifeunits'].sum(), action3] 
        Q[state2['Remaining_lifeunits'].sum(), action2] = Q[state2['Remaining_lifeunits'].sum(), action2] + alpha * (target - predict) 
        print(Q[state2['Remaining_lifeunits'].sum(), action2])
        
        inv_updated= action3                                        
        age_updated= 5                                   
        lifeunits_updated=inv_updated * age_updated
        state4_ini = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits']) 
        state4_ini.loc[0] = state3.loc[0]
        state4_ini.loc[1] = state3.loc[1]
        state4_ini.loc[2] = state3.loc[2]
        state4_ini.loc[3] = [inv_updated,age_updated,lifeunits_updated]
        print('\nstate4 initia lots :\n', state4_ini) 
        

        d4=np.random.gamma(4,5,1) 
        D4=int(d4)                                              
        print(D4)
        TS = max((D4-state4_ini['Inventory'].sum()),0)        
        
        state4 = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits'])
        state4.loc[0,'Inventory']= max((state4_ini.loc[0,'Inventory']-D4),0)  
        
        if state4_ini.loc[0,'Inventory']-D4<0:
            state4.loc[1,'Inventory']= max((state4_ini.loc[1,'Inventory']-(D4 - state4_ini.loc[0,'Inventory'])),0)
        else :
            state4.loc[1,'Inventory']=state4_ini.loc[1,'Inventory']
            
        if state4_ini.loc[1,'Inventory']-(D4 - state4_ini.loc[0,'Inventory'])<0: 
            state4.loc[2,'Inventory']= max((state4_ini.loc[2,'Inventory']-(D4 - (state4_ini.loc[1,'Inventory']+
                                                                             state4_ini.loc[0,'Inventory']))),0)
        else:    
            state4.loc[2,'Inventory']=state4_ini.loc[2,'Inventory']
            
        if state4_ini.loc[2,'Inventory']-(D4 - (state4_ini.loc[1,'Inventory']+ state4_ini.loc[0,'Inventory']))<0: 
                                           state4.loc[3,'Inventory']= max((state4_ini.loc[3,'Inventory']-(D4 - (state4_ini.loc[2,'Inventory']+state4_ini.loc[1,'Inventory']+state4_ini.loc[0,'Inventory']))),0)
        else:    
                                          state4.loc[3,'Inventory']=state4_ini.loc[3,'Inventory']
                
        state4.loc[0,'Age']= state4_ini.loc[0,'Age']-1 
        state4.loc[1,'Age']= state4_ini.loc[1,'Age']-1
        state4.loc[2,'Age']= state4_ini.loc[2,'Age']-1
        state4.loc[3,'Age']= state4_ini.loc[3,'Age']-1
        state4.loc[0,'Remaining_lifeunits']= state4.loc[0,'Inventory']*state4.loc[0,'Age']
        state4.loc[1,'Remaining_lifeunits']= state4.loc[1,'Inventory']*state4.loc[1,'Age']
        state4.loc[2,'Remaining_lifeunits']= state4.loc[2,'Inventory']*state4.loc[2,'Age']  
        state4.loc[3,'Remaining_lifeunits']= state4.loc[3,'Inventory']*state4.loc[3,'Age']
            
        print('\nstate4 lots :\n', state4)                     #state 4...for taking action 4
               
        outdated=state4[state4['Age']==0]
        print('\nOutdated dataframe :\n', outdated)                 
        TO=outdated['Inventory'].sum()
        print(TO)
        
        reward=-(Co*TO + Cs*TS)                              
        print('\nreward :\n', reward)
        action4 = choose_action(state4)         
        
        predict = Q[state3['Remaining_lifeunits'].sum(), action3]             #4. Q value update
        target = reward + gamma * Q[state4['Remaining_lifeunits'].sum(), action4] 
        Q[state3['Remaining_lifeunits'].sum(), action3] = Q[state3['Remaining_lifeunits'].sum(), action3] + alpha * (target - predict) 
        print(Q[state3['Remaining_lifeunits'].sum(), action3])
        
        inv_updated= action4                                         
        age_updated= 5                                   
        lifeunits_updated=inv_updated * age_updated
        
        #1. initial state in 5th timestep
        state5_ini = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits']) 
        state5_ini.loc[0] = state4.loc[0]
        state5_ini.loc[1] = state4.loc[1]
        state5_ini.loc[2] = state4.loc[2]
        state5_ini.loc[3] = state4.loc[3]
        state5_ini.loc[4] = [inv_updated,age_updated,lifeunits_updated]
        print('\nstate5 initia lots :\n', state5_ini) 
        
        #2.1demand occurs
        t=5
        d5=np.random.gamma(4,5,1) 
        D5=int(d5)                                              
        print(D5)
        TS = max((D5-state5_ini['Inventory'].sum()),0)        
        
        state5 = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits'])
        
        #2.2 FIFO policy
        state5.loc[0,'Inventory']= max((state5_ini.loc[0,'Inventory']-D5),0)  
        
        if state5_ini.loc[0,'Inventory']-D5<0:
            state5.loc[1,'Inventory']= max((state5_ini.loc[1,'Inventory']-(D5 - state5_ini.loc[0,'Inventory'])),0)
        else :
            state5.loc[1,'Inventory']=state5_ini.loc[1,'Inventory']
            
        if state5_ini.loc[1,'Inventory']-(D5 - state5_ini.loc[0,'Inventory'])<0: 
            state5.loc[2,'Inventory']= max((state5_ini.loc[2,'Inventory']-(D5 - (state5_ini.loc[1,'Inventory']+
                                                                             state5_ini.loc[0,'Inventory']))),0)
        else:    
            state5.loc[2,'Inventory']=state5_ini.loc[2,'Inventory']
            
        if state5_ini.loc[2,'Inventory']-(D5 - (state5_ini.loc[1,'Inventory']+ state5_ini.loc[0,'Inventory']))<0: 
                                           state5.loc[3,'Inventory']= max((state5_ini.loc[3,'Inventory']-(D5 - (state5_ini.loc[2,'Inventory']+state5_ini.loc[1,'Inventory']+state5_ini.loc[0,'Inventory']))),0)
        else:    
                                          state5.loc[3,'Inventory']=state5_ini.loc[3,'Inventory']
                
        if state5_ini.loc[3,'Inventory']-(D5 - (state5_ini.loc[2,'Inventory']+state5_ini.loc[1,'Inventory']+state5_ini.loc[0,'Inventory']))<0:
                                           state5.loc[4,'Inventory']= max((state5_ini.loc[4,'Inventory']-(D5 - (state5_ini.loc[3,'Inventory']+state5_ini.loc[2,'Inventory']+state5_ini.loc[1,'Inventory']+state5_ini.loc[0,'Inventory']))),0)
        else:    
                                          state5.loc[4,'Inventory']=state5_ini.loc[4,'Inventory']
                
        state5.loc[0,'Age']= state5_ini.loc[0,'Age']-1 
        state5.loc[1,'Age']= state5_ini.loc[1,'Age']-1
        state5.loc[2,'Age']= state5_ini.loc[2,'Age']-1
        state5.loc[3,'Age']= state5_ini.loc[3,'Age']-1
        state5.loc[4,'Age']= state5_ini.loc[4,'Age']-1
        state5.loc[0,'Remaining_lifeunits']= state5.loc[0,'Inventory']*state5.loc[0,'Age']
        state5.loc[1,'Remaining_lifeunits']= state5.loc[1,'Inventory']*state5.loc[1,'Age']
        state5.loc[2,'Remaining_lifeunits']= state5.loc[2,'Inventory']*state5.loc[2,'Age']  
        state5.loc[3,'Remaining_lifeunits']= state5.loc[3,'Inventory']*state5.loc[3,'Age']
        state5.loc[4,'Remaining_lifeunits']= state5.loc[4,'Inventory']*state5.loc[4,'Age']
        
        #3. final states in 5th timestep   
        print('\nstate5 lots :\n', state5)                     #state 5...for taking action 5
        
        outdated=state5[state5['Age']==0]
        print('\nOutdated dataframe :\n', outdated)                 
        TO=outdated['Inventory'].sum()
        print(TO)
        
        reward=-(Co*TO + Cs*TS)                              
        print('\nreward :\n', reward)
        action5 = choose_action(state5)     
        
        predict = Q[state4['Remaining_lifeunits'].sum(), action4]             #4. Q value update
        target = reward + gamma * Q[state5['Remaining_lifeunits'].sum(), action5] 
        Q[state4['Remaining_lifeunits'].sum(), action4] = Q[state4['Remaining_lifeunits'].sum(), action4] + alpha * (target - predict) 
        print(Q[state4['Remaining_lifeunits'].sum(), action4])
        print(Q)
        inv_updated= action5                                         
        age_updated= 5                                   
        lifeunits_updated=inv_updated * age_updated     
        
          #for i in range(6, Max_steps,1):
    for i in range(6, Max_steps,1):
        
        #1. initial state in ith timestep
        state = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits']) 
        state[i]_ini.loc[0] = state[i-1].loc[0]
        state[i]_ini.loc[1] = state[i-1].loc[1]
        state[i]_ini.loc[2] = state[i-1].loc[2]
        state[i]_ini.loc[3] = state[i-1].loc[3]
        state[i]_ini.loc[4] = [inv_updated,age_updated,lifeunits_updated]
        print('\nstate[i] initia lots :\n', state[i]_ini) 
        
        #2.1demand occurs
        d=np.random.gamma(4,5,1) 
        D=int(d)                                              
        print(D)
        TS = max((D-state[i]_ini['Inventory'].sum()),0)        
        
        state[i] = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits'])
        
        #2.2 FIFO policy
        state[i].loc[0,'Inventory']= max((state[i]_ini.loc[0,'Inventory']-D),0)  
        
        if state[i]_ini.loc[0,'Inventory']-D<0:
            state[i].loc[1,'Inventory']= max((state[i]_ini.loc[1,'Inventory']-(D - state[i]_ini.loc[0,'Inventory'])),0)
        else :
            state[i].loc[1,'Inventory']=state[i]_ini.loc[1,'Inventory']
            
        if state[i]_ini.loc[1,'Inventory']-(D - state[i]_ini.loc[0,'Inventory'])<0: 
            state[i].loc[2,'Inventory']= max((state[i]_ini.loc[2,'Inventory']-(D - (state[i]_ini.loc[1,'Inventory']+
                                                                             state[i]_ini.loc[0,'Inventory']))),0)
        else:    
            state[i].loc[2,'Inventory']=state[i]_ini.loc[2,'Inventory']
            
        if state[i]_ini.loc[2,'Inventory']-(D - (state[i]_ini.loc[1,'Inventory']+ state[i]_ini.loc[0,'Inventory']))<0: 
                                           state[i].loc[3,'Inventory']= max((state[i]_ini.loc[3,'Inventory']-(D - (state[i]_ini.loc[2,'Inventory']+state[i]_ini.loc[1,'Inventory']+state[i]_ini.loc[0,'Inventory']))),0)
        else:    
                                          state[i].loc[3,'Inventory']=state[i]_ini.loc[3,'Inventory']
                
        if state[i]_ini.loc[3,'Inventory']-(D - (state[i]_ini.loc[2,'Inventory']+state[i]_ini.loc[1,'Inventory']+state[i]_ini.loc[0,'Inventory']))<0:
                                           state[i].loc[4,'Inventory']= max((state[i]_ini.loc[4,'Inventory']-(D- (state[i]_ini.loc[3,'Inventory']+state[i]_ini.loc[2,'Inventory']+N_ini.loc[1,'Inventory']+N_ini.loc[0,'Inventory']))),0)
        else:    
                                          state[i].loc[4,'Inventory']=state[i]_ini.loc[4,'Inventory']
                
        state[i].loc[0,'Age']= state[i]_ini.loc[0,'Age']-1 
        state[i].loc[1,'Age']= state[i]_ini.loc[1,'Age']-1
        state[i].loc[2,'Age']= state[i]_ini.loc[2,'Age']-1
        state[i].loc[3,'Age']= state[i]_ini.loc[3,'Age']-1
        state[i].loc[4,'Age']= state[i]_ini.loc[4,'Age']-1
        state[i].loc[0,'Remaining_lifeunits']= state[i].loc[0,'Inventory']*state[i].loc[0,'Age']
        state[i].loc[1,'Remaining_lifeunits']= state[i].loc[1,'Inventory']*state[i].loc[1,'Age']
        state[i].loc[2,'Remaining_lifeunits']= state[i].loc[2,'Inventory']*state[i].loc[2,'Age']  
        state[i].loc[3,'Remaining_lifeunits']= state[i].loc[3,'Inventory']*state[i].loc[3,'Age']
        state[i].loc[4,'Remaining_lifeunits']= state[i].loc[4,'Inventory']*state[i].loc[4,'Age']
        
        #3. final states in 5th timestep   
        print('\nstate[i] lots :\n', state[i])                     #state i...for taking action i
        
        outdated=state[i][state[i]['Age']==0]
        print('\nOutdated dataframe :\n', outdated)                 
        TO=outdated['Inventory'].sum()
        print(TO)
        
        reward=-(Co*TO + Cs*TS)                              
        print('\nreward :\n', reward)
        action[i] = choose_action(state[i])  
        
        predict = Q[state[i]['Remaining_lifeunits'].sum(), action[i]]             #4. Q value update
        target = reward + gamma * Q[state[i]['Remaining_lifeunits'].sum(), action[i]] 
        Q[state[i]['Remaining_lifeunits'].sum(), action[i]] = Q[state[i]['Remaining_lifeunits'].sum(), action[i]] + alpha * (target - predict) 
        print(Q[state[i]['Remaining_lifeunits'].sum(), action[i]])
        
        inv_updated= action[i]                                         
        age_updated= 5                                   
        lifeunits_updated=inv_updated * age_updated     
 

        if episode==250000:
            Rewards.append(reward)
            print('Episode: {} Rewards: {}'.format(episode,reward))
            break
        
 
        
        
        
    
        
        
   



SyntaxError: invalid syntax (1510270809.py, line 280)

In [ ]:
     #for i in range(6, Max_steps,1):
    for i in range(6, Max_steps,1):
        
        #1. initial state in ith timestep
        state[i]_ini = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits']) 
        state[i]_ini.loc[0] = state[i-1].loc[0]
        state[i]_ini.loc[1] = state[i-1].loc[1]
        state[i]_ini.loc[2] = state[i-1].loc[2]
        state[i]_ini.loc[3] = state[i-1].loc[3]
        state[i]_ini.loc[4] = [inv_updated,age_updated,lifeunits_updated]
        print('\nstate[i] initia lots :\n', state[i]_ini) 
        
        #2.1demand occurs
        d=np.random.gamma(4,5,1) 
        D=int(d)                                              
        print(D)
        TS = max((D-state[i]_ini['Inventory'].sum()),0)        
        
        state[i] = pd.DataFrame(columns=['Inventory','Age','Remaining_lifeunits'])
        
        #2.2 FIFO policy
        state[i].loc[0,'Inventory']= max((state[i]_ini.loc[0,'Inventory']-D),0)  
        
        if state[i]_ini.loc[0,'Inventory']-D<0:
            state[i].loc[1,'Inventory']= max((state[i]_ini.loc[1,'Inventory']-(D - state[i]_ini.loc[0,'Inventory'])),0)
        else :
            state[i].loc[1,'Inventory']=state[i]_ini.loc[1,'Inventory']
            
        if state[i]_ini.loc[1,'Inventory']-(D - state[i]_ini.loc[0,'Inventory'])<0: 
            state[i].loc[2,'Inventory']= max(((state[i]_ini.loc[2,'Inventory'])-(D - (state[i]_ini.loc[1,'Inventory']+state[i]_ini.loc[0,'Inventory']))),0)
        else:    
            state[i].loc[2,'Inventory']=state[i]_ini.loc[2,'Inventory']
            
        if state[i]_ini.loc[2,'Inventory']-(D - (state[i]_ini.loc[1,'Inventory']+ state[i]_ini.loc[0,'Inventory']))<0: 
                                           state[i].loc[3,'Inventory']= max((state[i]_ini.loc[3,'Inventory']-(D - (state[i]_ini.loc[2,'Inventory']+state[i]_ini.loc[1,'Inventory']+state[i]_ini.loc[0,'Inventory']))),0)
        else:    
                                          state[i].loc[3,'Inventory']=state[i]_ini.loc[3,'Inventory']
                
        if state[i]_ini.loc[3,'Inventory']-(D - (state[i]_ini.loc[2,'Inventory']+state[i]_ini.loc[1,'Inventory']+state[i]_ini.loc[0,'Inventory']))<0:
                                           state[i].loc[4,'Inventory']= max((state[i]_ini.loc[4,'Inventory']-(D- (state[i]_ini.loc[3,'Inventory']+state[i]_ini.loc[2,'Inventory']+N_ini.loc[1,'Inventory']+N_ini.loc[0,'Inventory']))),0)
        else:    
                                          state[i].loc[4,'Inventory']=state[i]_ini.loc[4,'Inventory']
                
        state[i].loc[0,'Age']= state[i] _ini.loc[0,'Age']-1 
        state[i].loc[1,'Age']= state[i]_ini.loc[1,'Age']-1
        state[i].loc[2,'Age']= state[i]_ini.loc[2,'Age']-1
        state[i].loc[3,'Age']= state[i]_ini.loc[3,'Age']-1
        state[i].loc[4,'Age']= state[i]_ini.loc[4,'Age']-1
        state[i].loc[0,'Remaining_lifeunits']= state[i].loc[0,'Inventory']*state[i].loc[0,'Age']
        state[i].loc[1,'Remaining_lifeunits']= state[i].loc[1,'Inventory']*state[i].loc[1,'Age']
        state[i].loc[2,'Remaining_lifeunits']= state[i].loc[2,'Inventory']*state[i].loc[2,'Age']  
        state[i].loc[3,'Remaining_lifeunits']= state[i].loc[3,'Inventory']*state[i].loc[3,'Age']
        state[i].loc[4,'Remaining_lifeunits']= state[i].loc[4,'Inventory']*state[i].loc[4,'Age']
        
        #3. final states in 5th timestep   
        print('\nstate[i] lots :\n', state[i])                     #state i...for taking action i
        
        outdated=state[i][state[i]['Age']==0]
        print('\nOutdated dataframe :\n', outdated)                 
        TO=outdated['Inventory'].sum()
        print(TO)
        
        reward=-(Co*TO + Cs*TS)                              
        print('\nreward :\n', reward)
        action[i] = choose_action(state[i])  
        
        predict = Q[state[i]['Remaining_lifeunits'].sum(), action[i]]             #4. Q value update
        target = reward + gamma * Q[state[i]['Remaining_lifeunits'].sum(), action[i]] 
        Q[state[i]['Remaining_lifeunits'].sum(), action[i]] = Q[state[i]['Remaining_lifeunits'].sum(), action[i]] + alpha * (target - predict) 
        print(Q[state[i]['Remaining_lifeunits'].sum(), action[i]])
        
        inv_updated= action[i]                                         
        age_updated= 5                                   
        lifeunits_updated=inv_updated * age_updated     
 

        if episode==250000:
            Rewards.append(reward)
            print('Episode: {} Rewards: {}'.format(episode,reward))
            break
        
 
        
        
 
        